# Intalling Libraries

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install azure-storage-blob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 14.7 MB/s eta 0:00:00


# Imoporting Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy  as np
import warnings
warnings.filterwarnings('ignore')
import json
import re
import os
import time
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

## Extracting Data from NYC Open Data website

In [ ]:
#Azure connection
AZURE_STORAGE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=motorcollisions;AccountKey=Os5IgWkQ7zcYAOwCLBL+NpPXEpp+hRVNLoFRwyvfcnPTSqrpJykjygNsJQNBF7vLQYx7w5CeWW66+AStqsFyjg==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "nycmvs"

#Initiative Azure Blob Service Client
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

#For loop to divide the dataset by setting offset 50000
for i in range (1, 44, 1):
  URL = f'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=50000&$offset={i}'
  response = requests.get(URL)

  #check if response
  if response.status_code != 200:
        print(f"Error: Received status code {response.status_code} at offset {i}")
        break
  data = response.json()

  #break after the extracting is done
  if not data:
        print(f"No data returned at offset {i}. Stopping.")
        break


  #Convert the current chunk of data to JSON
  chunk_data = json.dumps(data)

  #generate a blob name for each chunk
  filename = f"motor_crashes_data_chunk_{i:03}.json"

  # Upload the JSON data as a blob to the container
  blob_client = container_client.get_blob_client(filename)
  blob_client.upload_blob(chunk_data, overwrite=True)
